<a href="https://colab.research.google.com/github/tanjimmostafa/Bias-detection-bangla-word-embeddings/blob/main/WEAT_r.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WEAT in Python2

This notebook collects the basic functions of the WEAT procedure.

Here are the data sets that are used to define the assciation concepts. Here ist is only positive and negative. Sets are named after the terminology in each source paper. Yet, concepts are similar.

In [ ]:
# prompt: pip install gensim

!pip install gensim

In [ ]:
pip install -U bnlp_toolkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 5.9 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171034 sha256=0d655d3b73dd64a2192e257a3c91f49886c95749b7340b9c97d8d2de5e672c94
  Stored in directory: /root/.cache/pip/wheels/31/8a/8c/315c9e5d7773f74b33d5ed33f075b49c6eaeb7cedbb86e2cf8
Successfully built emoji
  Attempting uninstall: sentencepiece
    Found existing installation: sentencepiece 0.1.99
    Uninstalling sentencepiece-0.1.99:
      Successfully uninstalled sentencepiece-0.1.99
  Attempting uni

In [ ]:
# prompt: locate in google drive

from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/Colab Notebooks'
!ls

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks
bnwiki_word2vec.model		    models		      util.py
foo.txt				    __pycache__		      WEAT-r.ipynb
GoogleNews-vectors-negative300.bin  task-spamfiltering.ipynb  wordSets.py


In [ ]:
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/util.py')
import util

In [ ]:
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/wordSets.py')
import wordSets

In [ ]:
# Association sets from IAT literature

# Greenwald et al.(1998) - Measuring Individual Differences In Implicit Cognition - The Implicit Association Test
gw_pos = ['caress', 'freedom', 'health', 'love', 'peace', 'cheer', 'friend', 'heaven', 'loyal', 'pleasure',
          'diamond', 'gentle', 'honest', 'lucky', 'rainbow', 'diploma', 'gift', 'honor', 'miracle', 'sunrise',
          'family', 'happy', 'laughter', 'paradise', 'vacation']

gw_neg = ('abuse crash filth murder sickness accident death grief poison stink assault disaster hatred pollute '
          'tragedy bomb divorce jail poverty ugly cancer evil kill rotten vomit agony prison').split()

# Nosek, Banaji, Greendwald (2002) - Math Male Me Female Therefore Me Not Math
Pleasant_1 = 'assertive athletic strong compassion support sympathetic laughter champion paradise vacation'.split()
Unpleasant_1 = 'brutal destroy ruthless confusion insecure naive bad poor waste crude'.split()

Pleasant_2 = 'ambition cuddle excitement glory joy love paradise pleasure romantic miracle'.split()
Unpleasant_2 = 'agony death detest disaster humiliate jealousy punishment stress tragedy war'.split()

Pleasant_3 = 'affectionate cozy enjoyment friend hug laughter passion peace snuggle triumph'.split()
Unpleasant_3 = 'afraid crucify despise failure hatred irritate nightmare slap terrible violent'.split()

# Nosek et al. (2002) - Harvesting  implicit  group  attitudes  and  beliefs  from  a demonstration  web  site
harvest_good = 'Joy Love Peace Wonderful Pleasure Friend Laughter Happy'.lower().split()
harvest_bad = 'Agony Terrible Horrible Nasty Evil War Awful Failure Death'.lower().split()

# Monteith & Pettit (2011) - Implicit and explicit  stigmatizing  attitudes  and  stereotypes  about  depression.
mp_good = 'positive pleasant enjoy glorious wonderful bliss'.split()
mp_bad = 'negative horrible agony terrible unpleasant despise'.split()

###
generalPos = set(Pleasant_1 + Pleasant_2 + Pleasant_3 + gw_pos + harvest_good + mp_good)
generalNeg = set(Unpleasant_1 + Unpleasant_2 + Unpleasant_3 + gw_neg + harvest_bad + mp_bad)

# further word sets:
from wordSets import *

In [ ]:
#For bnlp bn_wiki

!mkdir models
%cd models

mkdir: cannot create directory ‘models’: File exists
/content/drive/MyDrive/Colab Notebooks/models


In [ ]:
!wget https://huggingface.co/sagorsarker/bangla_word2vec/resolve/main/bangla_word2vec_gen4.zip
!unzip bangla_word2vec_gen4.zip
!rm -rf bangla_word2vec_gen4.zip

--2024-04-29 10:08:08--  https://huggingface.co/sagorsarker/bangla_word2vec/resolve/main/bangla_word2vec_gen4.zip
Resolving huggingface.co (huggingface.co)... 65.8.178.93, 65.8.178.12, 65.8.178.27, ...
Connecting to huggingface.co (huggingface.co)|65.8.178.93|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/f5/12/f51294acf565f8e7f2b231a61a327e6a79fce680615e3bbbc4850f166e928650/b157fe019ddda848f0195240f636ff2dcb63cbe823c84eed77c129e196105fd1?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27bangla_word2vec_gen4.zip%3B+filename%3D%22bangla_word2vec_gen4.zip%22%3B&response-content-type=application%2Fzip&Expires=1714644488&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNDY0NDQ4OH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy9mNS8xMi9mNTEyOTRhY2Y1NjVmOGU3ZjJiMjMxYTYxYTMyN2U2YTc5ZmNlNjgwNjE1ZTNiYmJjNDg1MGYxNjZlOTI4NjUwL2IxNTdmZTAxOWRkZGE4NDhmMDE

Start with some imports of course ...

In [ ]:
#from bnlp import BengaliWord2Vec as bwv
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
import numpy
import math
import logging, os
from util import *
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

Load pretrained W2V Google Model.
The models can be downloaded at https://github.com/mmihaltz/word2vec-GoogleNews-vectors and https://github.com/eyaler/word2vec-slim, respectively. In most cases, the slim version is more than sufficient and much faster.

In [ ]:
#m = KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary=True)
#m = KeyedVectors.load_word2vec_format('/content/drive/My Drive/Colab Notebooks/GoogleNews-vectors-negative300.bin', binary=True)
m = Word2Vec.load('/content/drive/My Drive/Colab Notebooks/models/bangla_word2vec/bnwiki_word2vec.model')
#m = textblob.Word2Vec.load('/content/drive/My Drive/Colab Notebooks/bnwiki_word2vec.model')


In [ ]:
# prompt: check codec type of this file

!file -bi /content/drive/My Drive/Colab Notebooks/models/bangla_word2vec/bnwiki_word2vec.model


cannot open `/content/drive/My' (No such file or directory)
cannot open `Drive/Colab' (No such file or directory)
cannot open `Notebooks/models/bangla_word2vec/bnwiki_word2vec.model' (No such file or directory)


Here are the basic functions. Functions are kept as modular as possible so that they can be replaced and manipulated straightforwardly.

In [ ]:
# import gensim
# from bnlp import BengaliWord2Vec as bwv
 # bwv = BengaliWord2Vec()

# model_path = "/content/drive/My Drive/Colab Notebooks/models/bangla_word2vec/bnwiki_word2vec.model"
# m = gensim.models.Word2Vec.load(model_path)
# methods = dir(bwv)
# print(methods)

# word = 'গ্রাম'
# vector = bwv.get_most_similar_words(m, word)
# print(vector)

# print(m.wv['গ্রাম'])

In [ ]:
# get most similar words to গ্রাম

vector = m.wv.most_similar('গ্রাম')
print(vector)


[('মৌজা', 0.7147208452224731), ('মহল্লা', 0.6878851652145386), ('তালুক', 0.6774986386299133), ('গ্রামের', 0.6585026383399963), ('তহসিল', 0.6413203477859497), ('পুরসভা', 0.6314382553100586), ('গ্রামে', 0.6264088749885559), ('সংগ্রামপুর', 0.6029990315437317), ('গ্রামাঞ্চল', 0.5972400307655334), ('মৌজার', 0.5972233414649963)]


In [ ]:
def get_cosines(model, word, word_list):
    # Convert the Word2Vec object to a list of words
    words = list(model.wv.key_to_index)

    """
    Calculate the cosine-similarity of a word to every word in a word list.
    Here it is the mean. May also be replaced by other metrices.
    :param model:       underlying model
    :param word:        target word
    :param word_list:   list of words to calculate distance to
    :return:            list of cosine similarities, mean cosine similarity
    """

    cosines = []
    for elem in word_list:
        try:
            cosines.append(model.similarity(word, elem))    # Similarity: Compute cosine similarity between two words.
        except KeyError:
            logging.info('no cosine for ' + word + ' and ' + elem + ' available')

    if not cosines:       # Fehler abfangen
        logging.error('No cosine values available')
        return 0, 0

    mean_cosine = sum(cosines) / len(cosines)
    return cosines, mean_cosine


def s_word(model, w, A, B, out):
    # Convert the Word2Vec object to a list of words
    words = list(model.wv.index_to_key)
    """
    Calculate the association of w with the attribute sets (Is w rather associated to A (positive value) or to B (negative value))
    :param model:       underlying model
    :param w:           target word
    :param A:           association set 1
    :param B:           association set 2
    :param out:         (boolean) only do prints if true
    :return:            s-value
    """
    # global words

    cosines_wA, mean_cos_wA = get_cosines(model, w, A)
    cosines_wB, mean_cos_wB = get_cosines(model, w, B)

    s = 0
    s_word_val = mean_cos_wA - mean_cos_wB          # negativ, wenn w zu B gehört, sonst positiv
    s += s_word_val
    if out:
        assignment = 'failed (0)'
        if s_word_val > 0:
            assignment = 'A (' + format(s_word_val, '.4f') + ')'
        elif s_word_val < 0:
            assignment = 'B (' + format(s_word_val, '.4f') + ')'

        info = w + ': A = ' + format(mean_cos_wA, '.4f') + ' ; B = ' \
               + format(mean_cos_wB, '.4f') + ' | assignment: ' + assignment
        print(info)
    return s_word_val


# not  used
def s(model, X, Y, A, B, out):
    # Convert the Word2Vec object to a list of words
    words = list(model.wv.index_to_key)

    """
    Calculate the differential association of the two sets of target group with the attribute. This is exactly the same
    function as "s_corrected" but without normalisation. Thus, this function is no longer used by default.
    :param model:       underlying model
    :param X:           target set 1
    :param Y:           target set 2
    :param A:           association set 1
    :param B:           association set 2
    :param out:         only do prints if true
    :return:            s value
    """

    # global words

    s_xAB_all = []
    s_yAB_all = []

    for x in X:
        if x not in model:
            logging.info(x + ' is not in vocabulary -> skip it')
        else:
            curr = s_word(model, x, A, B, out)             # im Optimalfall positiv
            s_xAB_all.append(curr)
    if out:
        print('~~~~~~~~~~*~~~~~~~~~~*~~~~~~~~~~*~~~~~~~~~~*~~~~~~~~~~*~~~~~~~~~~')

    for y in Y:
        if y not in words:
            logging.info(y + ' is not in vocabulary -> skip it')
        else:
            curr = s_word(words, y, A, B, out)             # im Optimalfall negativ
            s_yAB_all.append(curr)

    if not s_xAB_all or not s_yAB_all:          # Fehler abfangen
        logging.error('none of the words is in vocabulary')
    sum_sXAB = sum(s_xAB_all)                   # -> Summe positiv
    sum_sYAB = sum(s_yAB_all)                   # -> Summe negativ

    return sum_sXAB - sum_sYAB      # im Optimalfall ein hoher wert, da zwei Mal -


def s_corrected(model, X, Y, A, B, out):
    # Convert the Word2Vec object to a list of words
    words = list(model.wv.index_to_key)

    """
    Calculate the differential association of the two sets of target group with the attribute.
    This function is a corrected and thus updated version of s(model, X, Y, A, B, out).
    Normalisation allows to insert target word groups with different numbers of elements.
    :param model:       underlying model
    :param X:           target set 1
    :param Y:           target set 2
    :param A:           association set 1
    :param B:           association set 2
    :param out:         only do prints if true
    :return:            s value
    """
    # global words

    s_xAB_all = []
    s_yAB_all = []

    for x in X:
        if x not in words:
            logging.info(x + ' in vocabulary (s, x): ' + str(x in words))
            print(x, words, x in words)
        else:
            curr = s_word(words, x, A, B, out)          # im Optimalfall positiv
            s_xAB_all.append(curr)
    if out:
        print('~~~~~~~~~~*~~~~~~~~~~*~~~~~~~~~~*~~~~~~~~~~*~~~~~~~~~~*~~~~~~~~~~')

    for y in Y:
        if y not in words:
            logging.info(y + ' in vocabulary (s, y): ' + str(y in words))
            print(y, words, y in words)
        else:
            curr = s_word(words, y, A, B, out)          # im Optimalfall negativ
            s_yAB_all.append(curr)

    if not s_xAB_all or not s_yAB_all:                  # Fehler abfangen
        logging.error('none of the words is in vocabulary')
        return 0
    sum_sXAB = sum(s_xAB_all) / len(X)                  # -> Summe positiv
    sum_sYAB = sum(s_yAB_all) / len(Y)                  # -> Summe negativ

    return sum_sXAB - sum_sYAB                          # im Optimalfall ein hoher wert, da zwei Mal -


def effect_size(model, X, Y, A, B):
    # Convert the Word2Vec object to a list of words
    words = list(model.wv.index_to_key)

    """
    calculate the effect size of association
    :param X:           target set 1
    :param Y:           target set 2
    :param A:           association set 1
    :param B:           association set 2
    :param model:       underlying model
    :return:
    """
    # global words

    s_values_x = []
    s_values_y = []

    for x in X:
        if x not in words:
            logging.info(x + ' in vocabulary (effect_size, x): ' + str(x in words))
        else:
            s_values_x.append(s_word(words, x, A, B, False))
    for y in Y:
        if y not in words:
            logging.info(y + ' in vocabulary (effect_size, y): ' + str(y in words))
        else:
            s_values_y.append(s_word(words, y, A, B, False))

    if not s_values_x or not s_values_y:                # Fehler abfangen
        logging.error('non of the words is in vocabulary')
        return 0

    mean_s_val_x = sum(s_values_x) / len(s_values_x)
    mean_s_val_y = sum(s_values_y) / len(s_values_y)

    s_values_all = s_values_x + s_values_y
    mean_s_val_all = sum(s_values_all) / len(s_values_all)
    s_values_all_corrected = s_values_all
    s_values_all_corrected[:] = [((x - mean_s_val_all)**2) for x in s_values_all]        # iterable: (x_i - mean(x))²
    std_dev = math.sqrt((1 / (len(s_values_all) - 1)) * sum(s_values_all_corrected))

    return (mean_s_val_x - mean_s_val_y) / std_dev


def permutation_test(model, X, Y, A, B, tag, permutation_num):
    # Convert the Word2Vec object to a list of words
    words = list(model.wv.index_to_key)

    """
    do s(X,Y,A,B) for all possible permutations of X & Y to check whether they are lower than weat
    :param model:           underlying model
    :param X:               target set 1
    :param Y:               target set 2
    :param A:               association set 1
    :param B:               association set 2
    :param tag:             (string) name for the saved logfile (no file extension)
    :param permutation_num: Max number of random permutations
    :return:                weat value, p value, number of permutations with higher weat value,
                            ~ with equal ~, ~ with lower weat value
    """
    # global words
    all_target_words = X + Y
    n = len(all_target_words)
    print(n)
    if (n%2) == 1:
        logging.error('ungerade Anzahl von Target Words im Spiel!')
    all_permutations = []


    k = 0

    full = False
    if n <= 15:
        full = True
        logging.info('Do permutation test with full permutation.')
        combinations = list(itertools.combinations(all_target_words, len(X)))
        combinations = combinations[1:]               # remove second half and default order

        controlIter = 0
        for combination in combinations:
            controlIter += 1

            rest = all_target_words.copy()
            for elem in combination:
                rest.remove(elem)
            all_permutations.append([list(combination), rest])

    else:
        iterations = permutation_num
        logging.info('Too many elements for full permutation. Do random sampling. Iterations: ' + str(iterations))

        for _ in itertools.repeat(None, iterations):
            shuffle = numpy.random.permutation(all_target_words).tolist()
            half1 = shuffle[:n // 2]
            half2 = shuffle[n // 2:]
            all_permutations.append([half1,half2])
            k += 1
        # logging.info('permutations done. number: ' + str(len(all_permutations)))

    i = 0
    n_2 = len(all_permutations)
    higher = 0
    equal = 0
    lower = 0

    # logging.info('permutation test: call s-corrected')
    weat = s_corrected(words, X, Y, A, B, False)
    # logging.info('done: call s-corrected. result: ' + str(weat))
    plot_vals = []

    if not full:
        logging.info('betrachte Permutations-P-Werte nur im Betrag')
    logging.info('start loop')
    for permutation in all_permutations:
        s_val = s_corrected(words, permutation[0], permutation[1], A, B, False)
        if not full:
            s_val = abs(s_val)          # Betrachte nur Samples aus dem positiven Raum und versopple so die Anzahl
        plot_vals.append(s_val)
        bar(i, n_2, 50, "P value calculation: ")
        if s_val > weat:
            higher += 1
        elif weat > s_val:
            lower += 1
        elif weat == s_val:
            equal += 1
        else:
            logging.error('hier stimmt was nicht!')
        i += 1

    p_value = 1 - (lower / (higher + equal + lower))       # observed or greater difference
    if tag:
        plt_hist(plot_vals, tag, weat)
    return weat, p_value, higher, equal, lower


def plt_hist(list, name, weat=None, bins=30):
    """
    plot permutation stats as histogram to visualise significance of result. Automatically saves the plot
    :param list:        list of weat-values to plot
    :param name:        (string) name for log file (no file extension)
    :param weat:        (number) actual weat value
    :param bins:        bin size for histograms
    """
    plt.hist(list, bins)
    list.sort()
    if weat:
        plt.axvline(weat, color='r')
    #print(list)

    plt.title('permutation test values')
    plt.xlabel('value')
    plt.savefig('./plots/plt_' + name + '.pdf')


Call function executes the whole WEAT method for a certain set up.
model : pretrained W2V model; X,Y : Target sets; A,B : Association sets.

Hier ist ein bisschen Chaos, weil ich mir alle möglichen Sachen hab ausgeben lassen. Die erste der beiden call funktionen hat p-test signifikanzen enthalten. Die zweite ist etwas mehr basic.

In [ ]:
def call(model, X, Y, A, B, tag='none', lower_case=False, permutation_num=5000):

  # Convert the Word2Vec object to a list of words
  words = list(model.wv.index_to_key)


  if lower_case:
    logging.info('lower vocabulary')
    X = low(X)
    Y = low(Y)
    A = low(A)
    B = low(B)

    x_in = [x for x in X if x in words]
    x_not = [x for x in X if x not in words]

    y_in = [y for y in Y if y in words]
    y_not = [y for y in Y if y not in words]

    a_in = [a for a in A if a in words]
    a_not = [a for a in A if a not in words]

    b_in = [b for b in B if b in words]
    b_not = [b for b in B if b not in words]

    output = [tag + ': ']

    if not (x_in and y_in and a_in and b_in):
        output. append('one list is empty')
    elif len(x_in) < 3 or len(y_in) < 3:
        output.append('list to short')

    else:
        weat, p, h, e, l = permutation_test(model, x_in, y_in, a_in, b_in, tag, permutation_num)
        es = effect_size(model, X, Y, A, B)

        trans = ' '
        if p > 0.95:
            p = 1-p
            es = es * (-1)
            trans = ' T'

        if p < 0.05:
            if p < 0.01:
                if p < 0.001:
                    output.append(str(round(es, 4)) + trans + '***       ' + str(h+e+l))
                else:
                    output.append(str(round(es, 4)) + trans + '**        ' + str(h+e+l))
            else:
                output.append(str(round(es, 4)) + trans + '*         ' + str(h+e+l))
        else:
            output.append('nicht signifikant. '+ str(h+e+l))
    output.append(' \n')
    print(output)
    return output

In [ ]:
def call(model, X, Y, A, B, tag='none', lower_case=False, permutation_num=5000):
    """
    put everything together
    :param model:               underlying model
    :param X:                   target set 1
    :param Y:                   target set 2
    :param A:                   association set 1
    :param B:                   association set 2
    :param tag:                 name for saving, default='none'
    :param lower_case:
    :param permutation_num:     max number of permutation for permutation test
    :return:
    """
    # Convert the Word2Vec object to a list of words
    words = list(model.wv.index_to_key)


    if lower_case:
        print('#########################')
        logging.info('lower vocabulary')
        X = low(X)
        Y = low(Y)
        A = low(A)
        B = low(B)
    x_in = [x for x in X if x in words]
    x_not = [x for x in X if x not in words]
    y_in = [y for y in Y if y in words]
    y_not = [y for y in Y if y not in words]
    a_in = [a for a in A if a in words]
    a_not = [a for a in A if a not in words]
    b_in = [b for b in B if b in words]
    b_not = [b for b in B if b not in words]
    print(x_in, y_in, a_in, b_in)
    output = ['\n\n######################## ' + tag + ' ##### ' + str(lower_case) + ' ##################### \n']
    output.append('X : ' + str(x_in) + '\n')
    output.append('X negative : ' + str(x_not) + '\n')
    output.append('Y : ' + str(y_in) + '\n')
    output.append('Y negative : ' + str(y_not) + '\n\n')
    output.append('A: ' + str(a_in) + '\n')
    output.append('A negative: ' + str(a_not) + '\n')
    output.append('B: ' + str(b_in) + '\n')
    output.append('B negative: ' + str(b_not) + '\n\n')
    if not (x_in and y_in and a_in and b_in):
        logging.error('one list is empty')
        output.append('one list is empty!!!')
    else:
        weat, p, h, e, l = permutation_test(model, x_in, y_in, a_in, b_in, tag, permutation_num)
        es = effect_size(model, X, Y, A, B)
        output.append('weat value : ' + str(weat) + '\n')
        output.append('Permutation: p = ' + str(p) + ' (higher: ' + str(h) + ', equal: ' + str(e) + ', lower: ' + str(l) + ')\n')
        output.append('effect size: ' + str(es))
    return output

Now you can call single WEAT calculations. Postitiv WEAT value means A to X and B to Y. Negative WEAT value means A to Y and B to X.

In [ ]:
import wordSets
print(type(m))
print(len(wordSets.gw_flow))
print(len(wordSets.gw_insec))
print(call(m,wordSets.gw_flow, wordSets.gw_insec,generalPos,generalNeg, 'cd'))

<class 'gensim.models.word2vec.Word2Vec'>
25
25


ERROR:root:ungerade Anzahl von Target Words im Spiel!


['rose', 'pansy'] ['fly', 'cricket', 'moth'] ['health', 'family', 'support', 'honor', 'friend', 'honest', 'wonderful', 'peace', 'rainbow', 'positive', 'passion', 'diamond', 'glory', 'pleasure', 'lucky', 'freedom', 'strong', 'love', 'happy', 'heaven', 'gentle'] ['cancer', 'stress', 'evil', 'waste', 'jail', 'disaster', 'abuse', 'murder', 'negative', 'violent', 'poor', 'failure', 'war', 'death', 'kill', 'bad']
5


AttributeError: 'list' object has no attribute 'wv'